# **DEEP LEARNING MODEL WITH TENSORFLOW AND KERAS**
## **ABOUT THE DATA:**
### About Food Demand Forecasting Challenge
- Demand forecasting is a key component to every growing online business. Without proper demand forecasting processes in place, it can be nearly impossible to have the right amount of stock on hand at any given time. A food delivery service has to deal with a lot of perishable raw materials which makes it all the more important for such a company to accurately forecast daily and weekly demand.
 
- Too much inventory in the warehouse means more risk of wastage, and not enough could lead to out-of-stocks — and push customers to seek solutions from your competitors. In this challenge, get a taste of demand forecasting challenge using a real dataset.

### Problem Statement
- Your client is a meal delivery company which operates in multiple cities. They have various fulfillment centers in these cities for dispatching meal orders to their customers. The client wants you to help these centers with demand forecasting for upcoming weeks so that these centers will plan the stock of raw materials accordingly.

- The replenishment of majority of raw materials is done on weekly basis and since the raw material is perishable, the procurement planning is of utmost importance. Secondly, staffing of the centers is also one area wherein accurate demand forecasts are really helpful. Given the following information, the task is to predict the demand for the next 10 weeks (Weeks: 146-155) for the center-meal combinations in the test set:  

- Historical data of demand for a product-center combination (Weeks: 1 to 145)
    - Product(Meal) features such as category, sub-category, current price and discount
    - Information for fulfillment center like center area, city information etc.
 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = 15,8
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as splt
# for handling the outliers
from sklearn.ensemble import IsolationForest
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adadelta
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import load_model

## 1 IMPORTING THE DATA

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
print(train.head())

In [4]:
print(test.head())

## 2 FEATURE ENGINEERING
- **2.1. CHANGING THE DATA TYPE**

In [5]:
X = train.drop('id,week,num_orders,checkout_price,base_price'.split(','),axis=1)
Y = test.drop('id,week,checkout_price,base_price'.split(','),axis=1)

In [6]:
X = X.astype(str)
Y = Y.astype(str)

In [7]:
# print(X.describe())
# print(Y.describe())

In [8]:
train['center_id,meal_id,emailer_for_promotion,homepage_featured'.split(',')] = X
test['center_id,meal_id,emailer_for_promotion,homepage_featured'.split(',')] = Y

In [9]:
# train.describe(include='all')

In [10]:
# test.describe(include='all')

- **2.2 CREATING DUMMY VARIABLES**

In [11]:
dataset = train
for i in 'center_id,meal_id,emailer_for_promotion,homepage_featured'.split(','):
    data = pd.get_dummies(dataset[i],drop_first=True)
    dataset = pd.concat([dataset,data],axis=1)
train = dataset
train.drop('center_id,meal_id,emailer_for_promotion,homepage_featured'.split(','),axis=1,inplace=True)

In [12]:
dataset = test
for i in 'center_id,meal_id,emailer_for_promotion,homepage_featured'.split(','):
    data = pd.get_dummies(dataset[i],drop_first=True)
    dataset = pd.concat([dataset,data],axis=1)
test = dataset
test.drop('center_id,meal_id,emailer_for_promotion,homepage_featured'.split(','),axis=1,inplace=True)

In [13]:
# train.info()

In [14]:
# test.info()

## 4 EXCLUDING THE OUTLIERS

In [15]:
new_train = train.drop(['id','week'],axis=1)

In [16]:
# # for handling the outliers
# # from sklearn.ensemble import IsolationForest

# clf = IsolationForest(max_samples = 100, random_state = 42)
# clf.fit(new_train)
# y_noano = clf.predict(new_train)
# y_noano = pd.DataFrame(y_noano, columns = ['Top'])
# y_noano[y_noano['Top'] == 1].index.values

# new_train = new_train.iloc[y_noano[y_noano['Top'] == 1].index.values]
# new_train.reset_index(drop = True, inplace = True)
# print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
# print("Number of rows without outliers:", new_train.shape[0])

In [17]:
# new_train.info()

## 3 SEPARATING THE INDEPENDENT AND DEPENDENT VARIABLES

In [18]:
X = new_train.drop('num_orders',axis=1)
Y = new_train['num_orders']

In [19]:
# X.head()

In [20]:
new_test = test.drop(['id','week'],axis=1)

In [21]:
# new_test.head()

## 4 FEATURE SCALING

In [22]:
scale = StandardScaler()

In [23]:
data = X
# fitting and transforming the X into feature scaling
X = scale.fit_transform(X)
# converting again into dataframe
X = pd.DataFrame(X,columns=data.columns)
data = new_test
# fitting and transforming the test into feature scaling
new_test = scale.transform(new_test)
# converting again into dataframe
new_test = pd.DataFrame(new_test,columns=data.columns)

In [24]:
# X.shape

In [25]:
# new_test.shape

In [26]:
# print(X.head())
# print(new_test.head())

## 5 SPLITTING THE X DATA INTO TRAIN AND TEST

In [27]:
# X_train, X_test, Y_train, Y_test = splt(X, Y, test_size=0.9781, random_state=42)

In [28]:
# X_train.info()

## 6 SAMPLE MODEL

In [29]:
def build_regressor():
    regressor = Sequential()
    regressor.add(Dense(units=66,kernel_initializer='normal',
                     activation='relu',input_dim=130))
    regressor.add(Dense(units=66,kernel_initializer='normal',
                     activation='relu'))
    regressor.add(Dense(units=66,kernel_initializer='normal',
                     activation='relu'))
    regressor.add(Dense(units=66,kernel_initializer='normal',
                     activation='relu'))
    regressor.add(Dense(units=66,kernel_initializer='normal',
                     activation='relu'))
    regressor.add(Dense(units=1,kernel_initializer='normal',
                     activation='relu'))
    regressor.compile(optimizer=Adadelta(),loss='mean_squared_logarithmic_error')
    return regressor
regressor = KerasRegressor(build_fn= build_regressor,batch_size=10,epochs=100)

In [30]:
regressor.fit(x=X,y=Y)

Train on 456548 samples
Epoch 1/100
456548/456548 [==============================] - 326s 714us/sample - loss: 3.6188
Epoch 2/100
456548/456548 [==============================] - 281s 615us/sample - loss: 1.2137
Epoch 3/100
456548/456548 [==============================] - 281s 616us/sample - loss: 1.0186
Epoch 4/100
456548/456548 [==============================] - 281s 616us/sample - loss: 0.8317
Epoch 5/100
456548/456548 [==============================] - 281s 615us/sample - loss: 0.7031
Epoch 6/100
456548/456548 [==============================] - 291s 638us/sample - loss: 0.6234
Epoch 7/100
456548/456548 [==============================] - 334s 732us/sample - loss: 0.5713
Epoch 8/100
456548/456548 [==============================] - 283s 619us/sample - loss: 0.5375
Epoch 9/100
456548/456548 [==============================] - 279s 610us/sample - loss: 0.5152
Epoch 10/100
456548/456548 [==============================] - 285s 625us/sample - loss: 0.4997
Epoch 11/100
456548/456548 [=======

In [32]:
# regressor.fit(x=X_train,y=Y_train)

Train on 9998 samples
Epoch 1/100
9998/9998 [==============================] - 16s 2ms/sample - loss: 25.2349
Epoch 2/100
9998/9998 [==============================] - 13s 1ms/sample - loss: 25.0808
Epoch 3/100
9998/9998 [==============================] - 13s 1ms/sample - loss: 24.3701
Epoch 4/100
9998/9998 [==============================] - 13s 1ms/sample - loss: 17.7069
Epoch 5/100
9998/9998 [==============================] - 14s 1ms/sample - loss: 6.7432
Epoch 6/100
9998/9998 [==============================] - 13s 1ms/sample - loss: 3.3871
Epoch 7/100
9998/9998 [==============================] - 14s 1ms/sample - loss: 2.4505
Epoch 8/100
9998/9998 [==============================] - 9s 881us/sample - loss: 2.1045
Epoch 9/100
9998/9998 [==============================] - 7s 676us/sample - loss: 1.9350
Epoch 10/100
9998/9998 [==============================] - 6s 620us/sample - loss: 1.8264
Epoch 11/100
9998/9998 [==============================] - 7s 652us/sample - loss: 1.7448
Epoch 12/10

In [38]:
# parameters = {'batch_size': [10,100],
#              'epochs': [100,500]}

In [39]:
# grid_search = GridSearchCV(estimator=regressor,
#                           param_grid=parameters,
#                           cv=8)

In [65]:
# grid_search = grid_search.fit(X=X_train,y=Y_train)

In [83]:
# batch=100,epochs=200,genpact score=58
msle = cross_val_score(estimator=regressor, X=X_train, y=Y_train, cv=8, n_jobs= -1)

In [84]:
msle

array([-1.2108303 , -1.09141175, -1.11844567, -1.14719059, -1.16087241,
       -1.10070157, -1.05073974, -1.13713647])

In [34]:
# batch=10,epochs=100,genpact score=
msle2 = cross_val_score(estimator=regressor, X=X_train, y=Y_train, cv=8, n_jobs= -1)

In [35]:
msle2

array([-1.08299447, -1.07596189, -1.111733  , -1.05212329, -1.10924443,
       -1.09144832, -1.06011402, -1.11257951])

### With Week And Different Tuning get score 103

In [41]:
Y_pred = regressor.predict(new_test)

In [43]:
Y_pred

array([190.6174 , 196.05923, 167.09648, ..., 161.66206, 126.94097,
       153.99043], dtype=float32)

In [ ]:
# id,num_orders

In [44]:
submission = pd.DataFrame(Y_pred,columns=['num_orders'])

In [45]:
submission = pd.concat([test['id'],submission],axis=1)

In [46]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32573 entries, 0 to 32572
Data columns (total 2 columns):
id            32573 non-null int64
num_orders    32573 non-null float32
dtypes: float32(1), int64(1)
memory usage: 381.8 KB


In [47]:
submission.describe()

,id,num_orders
count,3.257300e+04,32573.000000
mean,1.248476e+06,123.600601
std,1.441580e+05,36.218822
min,1.000085e+06,44.302490
25%,1.123969e+06,94.348747
50%,1.247296e+06,120.853554
75%,1.372971e+06,148.965805
max,1.499996e+06,243.347885


In [48]:
submission.head()

,id,num_orders
0,1028232,190.617401
1,1127204,196.059235
2,1212707,167.096481
3,1082698,140.126144
4,1400926,119.282150


In [49]:
test[['id']].head()

,id
0,1028232
1,1127204
2,1212707
3,1082698
4,1400926


In [50]:
test.shape

(32573, 132)

In [51]:
submission.shape

(32573, 2)

In [52]:
submission.to_csv('submission.csv',index=False)

### New Predition (Y_pred2)

In [92]:
Y_pred2 = regressor.predict(new_test)

In [96]:
submission2 = pd.DataFrame(Y_pred2,columns=['num_orders'])

In [97]:
submission2 = pd.concat([test['id'],submission2],axis=1)

In [98]:
submission2.describe()

,id,num_orders
count,3.257300e+04,32573.000000
mean,1.248476e+06,207.850876
std,1.441580e+05,233.581116
min,1.000085e+06,3.809295
25%,1.123969e+06,51.215984
50%,1.247296e+06,123.923637
75%,1.372971e+06,277.776550
max,1.499996e+06,1996.601929


In [99]:
submission2.head()

,id,num_orders
0,1028232,218.885559
1,1127204,184.395081
2,1212707,101.682968
3,1082698,37.129036
4,1400926,32.776661


In [100]:
submission2.to_csv('submission2.csv',index=False)

## How To Save And Reload Model

- **SAVING THE MODEL**

In [38]:
# My model name is regressor and I am using model.save function to the model
# in HDF5 format.
# regressor.model.save('model.h5')

- **RELOADING THE MODEL**

In [52]:
# Reassigning the model after loading the models(model2,model3)
model = load_model('model3.h5')

## PREDICTIONS

- **PREDICTION 3**

In [34]:
Y_pred = model.predict(new_test)

In [49]:
submission = pd.DataFrame({'id':list(test['id']),
                           'num_orders':list(Y_pred.ravel())})

In [50]:
submission.head()

,id,num_orders
0,1028232,259.886139
1,1127204,158.027649
2,1212707,137.175491
3,1082698,26.660126
4,1400926,32.790962


In [51]:
submission.to_csv('submission3.csv',index=False)

- **PREDICTION 4**

In [53]:
Y_pred = model.predict(new_test)

In [54]:
submission = pd.DataFrame({'id':list(test['id']),
                           'num_orders':list(Y_pred.ravel())})

In [55]:
submission.to_csv('submission4.csv',index=False)